In [1]:
!nvidia-smi

Wed Jul 12 19:42:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install git+https://github.com/bofenghuang/vigogne

  Cloning https://github.com/bofenghuang/vigogne to /tmp/pip-req-build-7ljb80mh
  Running command git clone --filter=blob:none --quiet https://github.com/bofenghuang/vigogne /tmp/pip-req-build-7ljb80mh
  Resolved https://github.com/bofenghuang/vigogne to commit 2ad9c7cf1bbae93c188c3ec173623a50deb2dc8c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-tjfhbh8f/accelerate_519f7628d33e47e7938f88e687d3f26f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-tjfhbh8f/accelerate_519f7628d33e47e7938f88e687d3f26f
  Resolved https://github.com/huggingface/accelerate.git to commit e8c64f598b97fae55677abdf12b154cba37eaab1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [3]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from vigogne.preprocess import generate_inference_chat_prompt

In [5]:
base_model_name_or_path = "bofenghuang/vigogne-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)
# tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # trust_remote_code=True,
    # low_cpu_mem_usage=True,
)

# lora_model_name_or_path = ""
# model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
def infer(
    user_querys,
    temperature=0.1,
    max_new_tokens=512,
    **kwargs,
):
    prompt = generate_inference_chat_prompt(user_querys, tokenizer=tokenizer)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            max_new_tokens=max_new_tokens,
            **kwargs,
          ),
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

In [7]:
def chat(**kwargs):
    history = []
    while True:
        user_input = input(">> <|UTILISATEUR|>: ")
        model_response = infer([*history, [user_input, ""]], **kwargs)

        history.append([user_input, model_response])
        print(f">> <|ASSISTANT|>: {history[-1][1]}")

In [ ]:
chat()

>> <|UTILISATEUR|>: Explique la différence entre Dos et phishing.
>> <|ASSISTANT|>: Dos (Denial of Service) est une attaque informatique qui consiste à empêcher un utilisateur d'accéder à un service ou à un site web en utilisant une quantité excessive de trafic. Phishing, en revanche, est une technique utilisée par les pirates informatiques pour obtenir des informations personnelles ou financières d'une personne en lui envoyant un e-mail ou un message qui semble être de la part d'une entreprise légitime, mais qui est en réalité une tentative de vol d'identité ou de vol d'argent.
>> <|UTILISATEUR|>: Et DDoS ?
>> <|ASSISTANT|>: DDoS (Distributed Denial of Service) est une attaque informatique qui consiste à empêcher un utilisateur d'accéder à un service ou à un site web en utilisant une quantité excessive de trafic provenant de plusieurs sources. C'est une forme plus sophistiquée de Dos qui utilise des ordinateurs infectés par un logiciel malveillant pour lancer des attaques simultanées.